In [25]:
from database.market import Market
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [26]:
start = datetime(2011,1,1)

In [2]:
market = Market()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()

In [5]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [8]:
prices.columns

Index(['_id', 'date', 'close', 'high', 'low', 'open', 'volume', 'adjClose',
       'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor',
       'ticker'],
      dtype='object')

In [18]:
prices["date"] = pd.to_datetime(prices["date"])
prices["week"] = [x.week for x in prices["date"]]
prices["year"] = [x.year for x in prices["date"]]

In [19]:
aggregate = []
for ticker in tqdm(sp500["Symbol"].unique()):
    ticker_data = prices[prices["ticker"]== ticker]
    ticker_data = ticker_data.groupby(["year","week"]).mean()
    ticker_data["ticker"] = ticker
    aggregate.append(ticker_data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:49<00:00,  2.98it/s]


In [44]:
a = pd.concat(aggregate).reset_index()
final = a.pivot_table(values="adjClose",columns="ticker",index=["year","week"])

In [45]:
final.reset_index()

ticker,year,week,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,...,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL
0,1994,1,NaN,NaN,NaN,0.241071,NaN,NaN,3.9750,3.704921,...,NaN,7.770627,NaN,8.324440,NaN,6.055196,2.843749,6.844263,6.140897,NaN
1,1994,2,NaN,NaN,NaN,0.236010,NaN,NaN,3.7875,3.725424,...,NaN,7.712200,NaN,8.631575,NaN,6.013137,2.828881,6.924101,6.235891,NaN
2,1994,3,NaN,NaN,NaN,0.227954,NaN,NaN,3.9875,3.678304,...,NaN,7.934935,NaN,8.820915,NaN,5.963455,2.970473,6.998815,6.428971,NaN
3,1994,4,NaN,NaN,NaN,0.255310,NaN,NaN,4.1500,3.593590,...,NaN,8.063449,NaN,8.797988,NaN,6.034308,3.032537,7.041509,6.130498,NaN
4,1994,5,NaN,NaN,NaN,0.248557,NaN,NaN,3.9500,3.718907,...,NaN,8.209400,NaN,8.897603,NaN,6.115322,3.038437,7.110460,6.161975,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,2021,24,144.717348,22.626,196.167764,130.504000,114.9560,117.7300,307.8140,110.582000,...,54.3580,348.082000,23.970914,34.206000,125.108,68.196000,129.162000,62.578000,64.762288,115.6420
1433,2021,25,146.654822,22.266,200.922000,133.300000,114.2300,116.1880,319.4660,111.532000,...,53.4480,359.196000,23.348000,34.166000,125.706,66.696000,130.848000,63.964000,64.207387,116.5100
1434,2021,26,148.077768,21.334,206.988000,137.060000,113.4740,115.3460,313.5500,116.808000,...,52.9600,362.210000,23.052000,34.240000,121.720,66.500000,141.188000,63.028000,63.530000,119.8860
1435,2021,27,149.332500,20.640,209.780000,143.735000,116.3525,115.0075,327.7625,119.422500,...,52.0275,372.237500,23.110000,34.917500,113.835,67.482500,137.742500,60.787500,62.762500,120.3425


In [46]:
final.fillna(-99999,inplace=True)
final.reset_index(inplace=True)
final = final[final["year"] >= 2011]

missing_tickers = []
for col in final.columns:
    if col not in ["ticker","year","week"]:
        missing = final[final[col]==-99999]
        if missing.index.size > 1:
            missing_tickers.append(col)

for col in missing_tickers:
    final.drop(col,axis=1,inplace=True)

In [47]:
final.reset_index(inplace=True)
stuff = final.drop(["index","ticker"],axis=1,errors="ignore")

In [51]:
stuff

ticker,year,week,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,...,WMT,WRB,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY
0,2011,1,27.466078,10.370392,60.326427,10.223695,29.286127,9.9140,18.324017,39.460575,...,42.329358,14.752336,18.835567,13.498170,14.051925,80.746167,16.481682,23.836451,49.955970,32.626586
1,2011,2,28.344750,10.368507,60.533177,10.579683,29.788631,10.1740,18.174297,40.193922,...,42.436825,14.595661,18.623428,13.630519,14.949305,83.513820,16.418802,24.730459,50.892677,33.536310
2,2011,3,28.272738,9.480115,61.116713,10.278369,30.460051,9.7325,18.271802,41.297608,...,43.198825,14.957095,18.229722,13.674280,15.293370,83.290519,16.656350,25.111143,52.245107,33.788414
3,2011,4,27.940429,9.848197,61.821980,10.452770,30.803824,9.9260,18.021346,42.276626,...,44.372780,15.370393,18.427364,14.178415,16.050315,82.680258,16.720628,25.801584,52.738112,33.694869
4,2011,5,27.627278,9.082812,60.935082,10.554718,30.493155,11.0660,17.521584,42.653078,...,43.642319,15.509780,18.591722,14.655156,16.644355,84.043494,16.558536,26.437928,55.259095,33.772043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,2021,24,144.717348,22.626000,196.167764,130.504000,117.730000,307.8140,110.582000,284.372000,...,138.120000,74.413877,348.082000,23.970914,34.206000,125.108000,68.196000,129.162000,62.578000,64.762288
546,2021,25,146.654822,22.266000,200.922000,133.300000,116.188000,319.4660,111.532000,288.628000,...,136.966000,73.792467,359.196000,23.348000,34.166000,125.706000,66.696000,130.848000,63.964000,64.207387
547,2021,26,148.077768,21.334000,206.988000,137.060000,115.346000,313.5500,116.808000,297.992000,...,139.170000,74.944000,362.210000,23.052000,34.240000,121.720000,66.500000,141.188000,63.028000,63.530000
548,2021,27,149.332500,20.640000,209.780000,143.735000,115.007500,327.7625,119.422500,309.195000,...,139.885000,75.472500,372.237500,23.110000,34.917500,113.835000,67.482500,137.742500,60.787500,62.762500


In [52]:
market.connect()
market.store("weekly_average",stuff)
market.disconnect()